In [1]:
import sys, os
sys.path.append("../..")
import numpy as np
from scipy.constants import h, c, eV, nano, milli, degree, pi
import pyoptix
from pyoptix.classes import (Beamline, GaussianSource, OpticalElement,
                             ToroidalMirror, PlaneFilm)
from pyoptix.optimize import focus

hc = h*c/eV
pyoptix.output_notebook()

intialzing SR library
OptiX library initialized


Loading BokehJS ...

In [2]:
lambda_laser = 1080e-9 # m
def harmonique(N=21):
    return lambda_laser/N

# Definition des optiques 

In [3]:
helimag = Beamline(name="Helimag")
source = GaussianSource(name="Generation")
m1T = ToroidalMirror(name="M1")
m2T = ToroidalMirror(name="M2")
foc = PlaneFilm(name="Focalisation")
focM1 = PlaneFilm(name="FocalisationM1")
spectro = PlaneFilm(name="Spectrometre")

In [4]:
helimag.chains["source"] = [source, focM1]
helimag.chains["M1"] = [source, m1T, focM1]
helimag.chains["Complete"] = [source, m1T, m2T, foc, spectro]

# Definition des paramètres optiques

## Source 

In [5]:
source.nrays = 1000
source.sigma_x = 1.e-4 # 100µm
source.sigma_y = 1.e-4 # 100µm
source.sigma_x_div = 1.e-3 # 1 mrad
source.sigma_y_div = 1.e-3 # 1 mrad
source.next = m1T

In [6]:
# Calcul approximatif des rayons de courbures, on prend ftot = fm1/2
p = 1.80
q = p/10
f = 1/(1/p+1/q)
theta = 20*degree #rad
r = 2*f*np.sin(theta)/2.
R = 2*f/(2*np.sin(theta))
print(p, q, f, r, R)

1.8 0.18 0.16363636363636364 0.055966932544200335 0.478440720026687


In [7]:
m1T.distance_from_previous = 1.80 # m
m1T.theta = theta # rad
m1T.phi = 90*degree # rad
m1T.minor_curvature = 1/r # m-1
m1T.major_curvature = 1/R # m-1
m1T.next = m2T

In [8]:
m2T.distance_from_previous = 20e-3 # m
m2T.theta = theta # rad
m2T.phi = 0*degree # rad
m2T.minor_curvature = 1/r # m-1
m2T.major_curvature = 1/R # m-1
m2T.next = foc

In [9]:
foc.distance_from_previous = 40e-3
foc.next = spectro
foc.set_recording("input")

In [10]:
focM1.distance_from_previous = 60e-3
focM1.set_recording("input")

In [11]:
spectro.distance_from_previous = 50e-3
spectro.set_recording("input")

In [12]:
helimag.active_chain = "M1"

Chaîne M1:
	Generation -> M1 -> FocalisationM1 


In [13]:
helimag.clear_impacts(clear_source=True)
helimag.align(harmonique(21))
helimag.generate(harmonique(21))
helimag.radiate()

1

In [26]:
from pyoptix.optimize import slider_optimizer
slider_optimizer(m1T, "minor_curvature", (10, 100), 1, screen=focM1, beamline=helimag, 
                 wavelength=harmonique(21), display="yyp", light_spd=True, nrays=1000)

Helimag M1


interactive(children=(FloatSlider(value=10.0, continuous_update=False, description='minor_curvature', layout=L…

In [15]:
focM1.get_diagram(1000)

,X,Y,dX,dY,Lambda
0,-0.000913,-0.001051,0.034622,-0.009725,5.142857e-08
1,-0.000339,0.002574,0.014106,0.020306,5.142857e-08
2,0.000865,0.002226,-0.034853,0.016384,5.142857e-08
3,0.000192,0.001347,-0.007657,0.010632,5.142857e-08
4,-0.000153,0.001863,0.006223,0.014802,5.142857e-08
...,...,...,...,...,...
995,0.000106,0.004756,-0.004647,0.037946,5.142857e-08
996,-0.000609,-0.001105,0.022933,-0.009425,5.142857e-08
997,0.000495,0.000489,-0.019546,0.003452,5.142857e-08
998,0.000220,-0.002378,-0.007997,-0.018949,5.142857e-08


In [23]:
focM1.show_diagram(1000, display="xy", orthonorm=True)

None None


(ColumnDataSource(id='4913', ...),
 [<bokeh.io.notebook.CommsHandle at 0x1aad8ab4a60>])

In [17]:
f"{np.sqrt((1e-4)**2+(60*1e-3)**2)*2.35:.3e}"

'1.410e-01'